Importing necessary modules:

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from config import key as OMDBkey
from config import bigKey
import requests as req
import json
from pprint import pprint

key = bigKey
#key = OMDBkey
print(bigKey)

222e70ba


Some of the movie names have weird characters, so we needed to use different encoding to get it in.

In [2]:
movie_csv = "All U.S. Released Movies_ 1972-2016.csv"
main_df = pd.read_csv(movie_csv, encoding = "ISO-8859-1")
main_df.drop(['Created','Modified','Position','Description'],axis=1, inplace = True)
main_df.head()

,Const,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,tt0110912,Pulp Fiction,https://www.imdb.com/title/tt0110912/,movie,8.9,154.0,1994.0,"Crime, Drama",1782098.0,1994-05-21,Quentin Tarantino
1,tt1872181,The Amazing Spider-Man 2,https://www.imdb.com/title/tt1872181/,movie,6.6,142.0,2014.0,"Action, Adventure, Sci-Fi",409667.0,2014-04-10,Marc Webb
2,tt0111161,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,movie,9.3,142.0,1994.0,Drama,2281443.0,1994-09-10,Frank Darabont
3,tt0076759,Star Wars,https://www.imdb.com/title/tt0076759/,movie,8.6,121.0,1977.0,"Action, Adventure, Fantasy, Sci-Fi",1205171.0,1977-05-25,George Lucas
4,tt0088763,Back to the Future,https://www.imdb.com/title/tt0088763/,movie,8.5,116.0,1985.0,"Adventure, Comedy, Sci-Fi",1028721.0,1985-07-03,Robert Zemeckis


The .csv was great but didn't have all of the data we needed, luckily it had a column containing an unambiguous ID which we can use to request the OMDB API for missing data.


This is display the format of the json

In [3]:
ID = 'tt0076759'
call = f"http://www.omdbapi.com/?i={ID}&apikey={key}"
data = req.get(call).json()
pprint(data)

{'Actors': 'Mark Hamill, Harrison Ford, Carrie Fisher, Peter Cushing',
 'Awards': 'Won 6 Oscars. Another 52 wins & 28 nominations.',
 'BoxOffice': 'N/A',
 'Country': 'USA',
 'DVD': '21 Sep 2004',
 'Director': 'George Lucas',
 'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
 'Language': 'English',
 'Metascore': '90',
 'Plot': 'Luke Skywalker joins forces with a Jedi Knight, a cocky pilot, a '
         "Wookiee and two droids to save the galaxy from the Empire's "
         'world-destroying battle station, while also attempting to rescue '
         'Princess Leia from the mysterious Darth Vader.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzVlY2MwMjktM2E4OS00Y2Y3LWE3ZjctYzhkZGM3YzA1ZWM2XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg',
 'Production': '20th Century Fox',
 'Rated': 'PG',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.6/10'},
             {'Source': 'Rotten Tomatoes', 'Value': '92%'},
             {'Source': 'Metacritic', 'Value': '90/100'}],
 'Released':

![title](https://m.media-amazon.com/images/M/MV5BNzVlY2MwMjktM2E4OS00Y2Y3LWE3ZjctYzhkZGM3YzA1ZWM2XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg)

This is to create the columns that we need to fill.

In [4]:
main_df['Box_Office']=''
main_df['Rotten_Tomatoes_Rating']=''
main_df['Metacritic_Rating']=''
main_df['Rated']= ''
main_df['Home_Release']=''
main_df['Production']=''
main_df['Country']=''
main_df['Awards_Blurb']=''
main_df['Languages']=''

Actually pulling all of the data from the jsons.
Getting all of the data for a particular row in one pull make the code a little busy and hard to break up in digestible pieces:

In [ ]:
count = 0
for index, row in main_df.iterrows():
    count += 1
    try:
        print(f"#{count}: Getting data for {row['Title']}")
        ID = row['Const']
        call = f"http://www.omdbapi.com/?i={ID}&apikey={key}"
        data = req.get(call).json()
        main_df.loc[index,'Box_Office'] = data['BoxOffice']
        main_df.loc[index, 'Rated'] = data['Rated']
        main_df.loc[index, 'Production'] = data['Production']
        main_df.loc[index, 'Country'] = data['Country']
#Rotten Tomatoes rating information is in a stupid format
#Here is how we had to pull it out:
        if len(data['Ratings'])>0:
            for i in data['Ratings']:
                if i['Source'] == 'Rotten Tomatoes':
                    main_df.loc[index,'Rotten_Tomatoes_Rating'] = i['Value']
                if i['Source'] == 'Metacritic':
                    main_df.loc[index,'Metacritic_Rating'] = i['Value']
        try:
            main_df.loc[index, 'Home_Release'] = data['DVD']
        except:
            main_df.loc[index, 'Home_Release'] = np.NaN
        main_df.loc[index, 'Awards_Blurb'] = data['Awards']
        main_df.loc[index, 'Languages'] = data['Language']
    except:
        print(f"Unable to get data for {row['Title']}")

The data in some of the columns needed to be cleaned up.  Here is where those functions were constructed:

In [10]:
#This cleans up cells starting with a '$' and handles numbers with commas
def dollarCleaner(x):
    try:
        x = x.replace('$','')
        x = x.replace(',','')
        x = int(x)
    except:
        x = np.NaN
    return x

#This one is made for Rotten Tomatoes ratings, It removes percent signs from the beginning
def lessRottenTomato(x):
    try:
        x = x.replace('%','')
        x = int(x)
    except:
        x = np.NaN
    return x

# This one is designed for metacritic and will take the top number from "top/bottom" format
def megacritic(x):
    try:
        x = x.split('/')
        x = x[0]
        x = int(x)
    except:
        x = np.NaN
    return x

#This will extract the numerical month as long as it is the center value in a string separated by '-'
def monthGetter(x):
    try:
        x = x.split('-')
        x = x[1]
    except:
        x = np.NaN
    try:
        x = int(x)
    except:
        x = np.NaN
    return x

#This one is designed to split the values in the 'Genres' column into lists, which we thought might be useful
def genreSplitter(x):
    y = []
    try:
        y = x.split(',')
    except:
        y.append(x)
    return y
#This one is to put the date in 'Home Release' into the same fomat as the other dates
#(turning 'May' into '5' will be another story)
def dateFormatter(x):
    y=0
    try:
        x = x.split(' ')
        y = f"{x[2]}-{x[1]}-{x[0]}"
        return y
    except:
        return y

This part applies htese cleaners to their respective rows:

In [7]:
main_df['Box_Office'] = main_df['Box_Office'].apply(dollarCleaner)
main_df['Rotten_Tomatoes_Rating'] = main_df['Rotten_Tomatoes_Rating'].apply(lessRottenTomato)
main_df['Metacritic_Rating'] = main_df['Metacritic_Rating'].apply(megacritic)
main_df['Month Released'] = main_df['Release Date'].apply(monthGetter)
main_df['Genres']= main_df['Genres'].apply(genreSplitter)
main_df['Home_Release']= main_df['Home_Release'].apply(dateFormatter)
main_df['Languages']= main_df['Languages'].apply(genreSplitter)
main_df.head()

,Const,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,...,Box_Office,Rotten_Tomatoes_Rating,Metacritic_Rating,Rated,Home_Release,Production,Country,Awards_Blurb,Languages,Month Released
0,tt0110912,Pulp Fiction,https://www.imdb.com/title/tt0110912/,movie,8.9,154.0,1994.0,"[Crime, Drama]",1782098.0,1994-05-21,...,NaN,91.0,94.0,R,1998-May-19,Miramax Films,USA,Won 1 Oscar. Another 69 wins & 74 nominations.,"[English, Spanish, French]",5.0
1,tt1872181,The Amazing Spider-Man 2,https://www.imdb.com/title/tt1872181/,movie,6.6,142.0,2014.0,"[Action, Adventure, Sci-Fi]",409667.0,2014-04-10,...,183277573.0,51.0,53.0,PG-13,2014-Aug-19,Sony Pictures,USA,4 wins & 30 nominations.,[English],4.0
2,tt0111161,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,movie,9.3,142.0,1994.0,[Drama],2281443.0,1994-09-10,...,NaN,90.0,80.0,R,1998-Jan-27,Columbia Pictures,USA,Nominated for 7 Oscars. Another 21 wins & 35 n...,[English],9.0
3,tt0076759,Star Wars,https://www.imdb.com/title/tt0076759/,movie,8.6,121.0,1977.0,"[Action, Adventure, Fantasy, Sci-Fi]",1205171.0,1977-05-25,...,NaN,92.0,90.0,PG,2004-Sep-21,20th Century Fox,USA,Won 6 Oscars. Another 52 wins & 28 nominations.,[English],5.0
4,tt0088763,Back to the Future,https://www.imdb.com/title/tt0088763/,movie,8.5,116.0,1985.0,"[Adventure, Comedy, Sci-Fi]",1028721.0,1985-07-03,...,2925880.0,96.0,87.0,PG,2002-Dec-17,Universal Pictures,USA,Won 1 Oscar. Another 19 wins & 25 nominations.,[English],7.0


One of our original ideas required the number of nominations for awards a particular movie had.  The sentence was difficult to parse, but here is one way we navigated it:

In [ ]:
def award_gen(x):
    y = {}
    try:
        x=x.split(" ")
        for i in range(len(x)):
                try:
                    x[i] = int(x[i])
                    y[x[i+1]] = x[i]
                except:
                    pass
        if x[0] == 'Nominated':
            y['nominations.'] += int(x[2])
        return y
    except:
        return "Could not parse awards"

        
main_df['Awards'] = main_df['Awards_Blurb'].apply(award_gen)
main_df.head()

Now just to remove duplicates remove movies from other countries which were often too different to lump together,
then save the output as its own csv for later work:

In [17]:
main_df = main_df.groupby('Const').first()
main_df = main_df[main_df['Country']=='USA']
main_df.to_csv('movies.csv')
main_df

,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors,...,Rotten_Tomatoes_Rating,Metacritic_Rating,Rated,Home_Release,Production,Country,Awards_Blurb,Languages,Month Released,Awards
Const,,,,,,,,,,,,,,,,,,,,,
tt0035423,Kate & Leopold,https://www.imdb.com/title/tt0035423/,movie,6.4,118.0,2001.0,"[Comedy, Fantasy, Romance]",77905.0,2001-12-11,James Mangold,...,50.0,44.0,PG-13,2002-Jun-11,Miramax Films,USA,Nominated for 1 Oscar. Another 1 win & 4 nomin...,"[English, French]",12.0,"{'Oscar.': 1, 'win': 1, 'nominations.': 5}"
tt0065643,The Dirty Mind of Young Sally,https://www.imdb.com/title/tt0065643/,movie,5.2,95.0,1973.0,[Comedy],226.0,1973-04-20,Bethel Buckalew,...,NaN,NaN,X,0,N/A,USA,N/A,[English],4.0,{}
tt0067716,Schlock,https://www.imdb.com/title/tt0067716/,movie,5.7,80.0,1973.0,"[Comedy, Horror, Sci-Fi]",1221.0,1973-04-11,John Landis,...,71.0,NaN,PG,2001-Oct-02,Jack H. Harris Enterprises,USA,1 win & 1 nomination.,[English],4.0,"{'win': 1, 'nomination.': 1}"
tt0068156,1776,https://www.imdb.com/title/tt0068156/,movie,7.6,141.0,1972.0,"[Drama, Family, History, Musical]",7588.0,1972-11-09,Peter H. Hunt,...,69.0,NaN,G,2002-Jul-02,Sony Pictures Home Entertainment,USA,Nominated for 1 Oscar. Another 1 win & 1 nomin...,[English],11.0,Could not parse awards
tt0068168,Across 110th Street,https://www.imdb.com/title/tt0068168/,movie,7.0,102.0,1972.0,"[Action, Crime, Drama, Thriller]",4923.0,1972-12-19,Barry Shear,...,81.0,NaN,R,2001-Oct-16,MGM,USA,N/A,"[English, Italian]",12.0,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt4046784,Maze Runner: The Scorch Trials,https://www.imdb.com/title/tt4046784/,movie,6.3,131.0,2015.0,"[Action, Adventure, Sci-Fi, Thriller]",216334.0,2015-09-01,Wes Ball,...,47.0,43.0,PG-13,2015-Dec-15,20th Century Fox,USA,2 wins & 5 nominations.,[English],9.0,"{'wins': 2, 'nominations.': 5}"
tt4120636,Green Lantern Corps,https://www.imdb.com/title/tt4120636/,movie,NaN,NaN,NaN,"[Action, Adventure, Sci-Fi]",NaN,NaN,NaN,...,NaN,NaN,N/A,0,Warner Bros. Pictures,USA,N/A,[English],NaN,{}
tt4121026,Justice League Part Two,https://www.imdb.com/title/tt4121026/,movie,NaN,NaN,NaN,"[Action, Sci-Fi]",NaN,NaN,NaN,...,NaN,NaN,N/A,0,N/A,USA,N/A,[English],NaN,{}
